### Paquetes necesarios

In [3]:
import cv2
import os
from deepface import DeepFace

### Carga de imágenes

In [4]:
# Tamaño deseado para las imágenes redimensionadas
desired_size = (256, 256)

# Carga de imágenes y asociado de etiquetas
image1 = {"img": cv2.imread("./images/happy.jpg"), "label": "happy"}
image1["img"] = cv2.resize(image1["img"], desired_size)

image2 = {"img": cv2.imread("./images/happy1.png"), "label": "happy"}
image2["img"] = cv2.resize(image2["img"], desired_size)

image3 = {"img": cv2.imread("./images/happy3.jpg"), "label": "happy"}
image3["img"] = cv2.resize(image3["img"], desired_size)

image4 = {"img": cv2.imread("./images/angry.jpeg"), "label": "angry"}
image4["img"] = cv2.resize(image4["img"], desired_size)

image5 = {"img": cv2.imread("./images/angry2.jpg"), "label": "angry"}
image5["img"] = cv2.resize(image5["img"], desired_size)

image6 = {"img": cv2.imread("./images/sad.jpg"), "label": "sad"}
image6["img"] = cv2.resize(image6["img"], desired_size)

# Lista que almacena las imágenes
images = [image1, image2, image3, image4, image5, image6]

In [5]:
# Contador de aciertos
counter = 0

# Contador de frames
frame_counter = 0

# Índice de la imagen actual
current_image_index = 0

# Mensaje de fallo o acierto
result_message = ""

# Color acierto o fallo
result_color = (0, 0, 0)

# Inicia la captura de video desde la webcam
cap = cv2.VideoCapture(0)

while True:
    # Captura el frame de la webcam
    ret, frame = cap.read()

    # Extrae coordenadas de la cara cada 5 frames
    if frame_counter % 5 == 0:
        try: 
                faces = DeepFace.extract_faces(frame, detector_backend='opencv')
                x = faces[0]['facial_area']['x']
                y = faces[0]['facial_area']['y']
                w = faces[0]['facial_area']['w']
                h = faces[0]['facial_area']['h']
        except:
            continue

    # Si se terminan las imágenes, muestra el resultado final
    if current_image_index >= len(images):
        cv2.putText(frame, f"Fin del juego", (x - 10, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (0, 234, 253), 2)
        cv2.putText(frame, f"Puntuacion final: {counter}/{len(images)}", (x - 10, y), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (119, 253, 0), 2)
        cv2.imshow("Webcam", frame)
    else:
        # Muestra el contador en la esquina superior izquierda del frame
        cv2.putText(frame, f"Contador = {counter}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Muestra si la emoción detectada es correcta o no
        cv2.putText(frame, f"{result_message}", (x + w//2 - 70, y), cv2.FONT_HERSHEY_SIMPLEX, 1, result_color, 2)

        # Muestra el frame en una ventana
        cv2.imshow("Webcam", frame)

        # Muestra la imagen flotante actual
        cv2.imshow("Random Image", images[current_image_index]["img"])

        # Si se han mostrado 100 frames, analiza la emoción
        if frame_counter == 100:
            # Utiliza DeepFace para analizar la emoción en el siguiente frame
            try:
                result = DeepFace.analyze(frame, actions = ['emotion'])
                emotion = result[0]["dominant_emotion"]
                print("Se ha detectado la emoción: " + emotion)

                # Si la emoción detectada coincide con la etiqueta de la imagen actual
                if emotion == images[current_image_index]["label"]:
                    counter += 1
                    result_message = "Correcto"
                    result_color = (0, 255, 0)
                else:
                    result_message = " Fallo" 
                    result_color = (0, 0, 255)    
            except:
                continue

        # Si se han mostrado 150 frames, pasa a la siguiente imagen
        if frame_counter == 200:
            current_image_index += 1
            result_message = ""
            frame_counter = 0
        
    # Se incrementa el contador de frames
    frame_counter += 1

    # Espera 1 milisegundo para la entrada de teclado
    key = cv2.waitKey(1)

    # Esc para finalizar
    if key == 27:  # Esc
        break

# Libera la captura de video y cierra todas las ventanas
cap.release()
cv2.destroyAllWindows()